# Import et traitement des données

In [1]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
from shapely import line_merge, MultiLineString, LineString
from shapely.geometry import Point, MultiPoint
from shapely.geometry import Polygon, MultiPolygon
import os
from shapely.wkt import loads
from google.colab import drive
import ast
import folium
from shapely.wkt import loads
import branca.colormap as cm
from folium.plugins import MeasureControl

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
Chemin_Donnees_trafic : str = "/content/drive/MyDrive/PIP2025_G8/Donnees/DonneesBrutes/Trafic/2019/routes_fusionnees_tmja.csv"
df_trafic = pd.read_csv(Chemin_Donnees_trafic)
#df_trafic.columns

In [49]:
Chemin_Donnees_bornes : str = "/content/drive/MyDrive/PIP2025_G8/Donnees/DonneesBrutes/Bornes_recharge/consolidation-etalab-schema-irve-statique-v-2.3.1-20250105_bis.csv"
df_bornes = pd.read_csv(Chemin_Donnees_bornes)
#df_bornes.columns

<ipython-input-49-e3a19e5e9c0b>:2: DtypeWarning: Columns (13,19,20,21,22,23,24,25,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bornes = pd.read_csv(Chemin_Donnees_bornes)


In [50]:
#Préparation du geodataframe trafic
df_trafic['geometry'] = df_trafic['geometry'].apply(
    lambda geom: loads(geom) if isinstance(geom, str) else geom
)
gdf_trafic = gpd.GeoDataFrame(df_trafic, geometry=df_trafic['geometry'], crs="EPSG:2154")

#Préparation du geodataframe bornes
df_bornes['geometry'] = df_bornes.apply(
    lambda row: Point(row['consolidated_longitude'], row['consolidated_latitude']), axis=1
)
gdf_bornes = gpd.GeoDataFrame(df_bornes, geometry=df_bornes['geometry'], crs="EPSG:4326").to_crs("EPSG:2154")

#Passage en Lambert 93
gdf_bornes['consolidated_longitude'] = gdf_bornes.geometry.x
gdf_bornes['consolidated_latitude'] = gdf_bornes.geometry.y

# Calcul des positions des bornes à ajouter

In [52]:
#Fonction déterminant les positions des bornes à ajouter pour une seule section
def ajouter_bornes_par_section(section_geometry, bornes_coords, x, longueur_min=30000):
    """
    Cette fonction calcule les positions supplémentaires de bornes à installer
    le long d'une section de route, afin de garantir une distance maximale de x
    entre les bornes existantes et celles à ajouter. Elle peut gérer des
    sections représentées par des géométries de type LineString ou MultiLineString.

    Paramètres :
    - section_geometry : (LineString ou MultiLineString) La géométrie de la section de route.
      Elle peut être une chaîne de caractères à convertir en géométrie ou un objet Shapely.
    - bornes_coords : (list) Une liste de coordonnées (longitude, latitude) représentant
      les positions des bornes existantes.
    - x : (float) La distance maximale en mètres entre deux bornes consécutives.
    - longueur_min : (float) La longueur minimale d'une section pour laquelle
      une borne doit être ajoutée si aucune borne n'existe.

    Fonctionnement :
    1. Si la section est de type MultiLineString, chaque composante (LineString) est traitée
       séparément en appelant la fonction de façon récursive.
    2. Pour une section de type LineString :
       - Les bornes existantes sont projetées sur la géométrie pour calculer leur position
         le long de la section.
       - Si aucune borne n'est présente et que la longueur de la section dépasse
         longueur_min, une borne est ajoutée au milieu de la section.
       - Si des bornes existent, la fonction calcule les positions supplémentaires
         nécessaires pour respecter la distance maximale (x) entre elles.
    3. La fonction retourne une liste de positions (longitude, latitude) des bornes
       supplémentaires à ajouter.

    Retour :
    - Une liste de tuples représentant les coordonnées des bornes supplémentaires à ajouter.
    """

    if isinstance(section_geometry, str):
        section_geometry = loads(section_geometry)

    if not isinstance(section_geometry, (LineString, MultiLineString)):
        raise ValueError(f"La géométrie n'est pas un LineString ou MultiLineString valide : {section_geometry}")

    positions_a_ajouter = []

    #Gère les MultiLineString
    if isinstance(section_geometry, MultiLineString):
        for line in section_geometry.geoms:
            positions_a_ajouter.extend(ajouter_bornes_par_section(line, bornes_coords, x, longueur_min))
        return positions_a_ajouter

    #Conversion de types et tris des bornes existantes
    bornes_points = [Point(coord) for coord in bornes_coords]
    bornes_projetees = [section_geometry.interpolate(section_geometry.project(borne)) for borne in bornes_points]
    bornes_sorted = sorted(bornes_projetees, key=lambda p: section_geometry.project(p))

    #Sections sans bornes
    if not bornes_sorted:
        if section_geometry.length >= longueur_min:
            midpoint = section_geometry.interpolate(section_geometry.length / 2)
            return [(midpoint.x, midpoint.y)]
        else:
            return []

    #Traite la partie entre le début de la section et la première borne
    if section_geometry.project(bornes_sorted[0]) > x:
        n_bornes = int(section_geometry.project(bornes_sorted[0]) // x)
        for i in range(n_bornes):
            new_position = section_geometry.interpolate((i + 1) * x)
            positions_a_ajouter.append((new_position.x, new_position.y))

    #Traite le cas classique de distance entre deux bornes consécutives
    for i in range(len(bornes_sorted) - 1):
        point1 = bornes_sorted[i]
        point2 = bornes_sorted[i + 1]
        position1 = section_geometry.project(point1)
        position2 = section_geometry.project(point2)
        distance = position2 - position1

        if distance > x:
            n_bornes = int(distance // x)
            segment_length = distance / (n_bornes + 1)
            for j in range(1, n_bornes + 1):
                new_position = section_geometry.interpolate(position1 + j * segment_length)
                positions_a_ajouter.append((new_position.x, new_position.y))

    #Traite la partie entre la dernière borne et la fin de la section
    if section_geometry.length - section_geometry.project(bornes_sorted[-1]) > x:
        distance_to_end = section_geometry.length - section_geometry.project(bornes_sorted[-1])
        n_bornes = int(distance_to_end // x)
        segment_length = distance_to_end / (n_bornes + 1)
        for i in range(1, n_bornes + 1):
            new_position = section_geometry.interpolate(section_geometry.project(bornes_sorted[-1]) + i * segment_length)
            positions_a_ajouter.append((new_position.x, new_position.y))

    return positions_a_ajouter


def ajouter_bornes(df_trafic, df_bornes, x, longueur_min=30000):
    """
    Cette fonction identifie les positions supplémentaires de bornes à installer
    pour l'ensemble des sections de route contenues dans un DataFrame. Elle garantit
    que la distance maximale entre deux bornes (existantes ou ajoutées) ne dépasse
    pas une valeur donnée.

    Paramètres :
    - df_trafic : (DataFrame) Un DataFrame contenant les informations sur les sections
      de route, incluant les colonnes 'route', 'geometry', et 'borne_ids_proches'.
    - df_bornes : (DataFrame) Un DataFrame contenant les informations sur les bornes
      existantes, incluant leurs coordonnées et leurs identifiants.
    - x : (float) La distance maximale en mètres entre deux bornes consécutives.
    - longueur_min : (float) La longueur minimale d'une section pour laquelle une borne
      doit être ajoutée si aucune borne n'existe.

    Fonctionnement :
    1. La colonne 'borne_ids_proches' est convertie en liste si elle est stockée
       sous forme de chaîne de caractères.
    2. Chaque section de route (ligne de df_trafic) est traitée individuellement :
       - La géométrie de la section (LineString ou MultiLineString) est extraite.
       - Les identifiants des bornes associées à la section ('borne_ids_proches')
         sont utilisés pour filtrer les bornes correspondantes dans df_bornes.
       - Les coordonnées des bornes associées sont récupérées.
    3. La fonction `ajouter_bornes_par_section` est appelée pour chaque section
       afin de calculer les positions supplémentaires de bornes à ajouter.
    4. Les positions calculées pour chaque section sont enregistrées dans un
       dictionnaire avec l'identifiant de la section ('route') comme clé.

    Retour :
    - Un dictionnaire où les clés sont les identifiants des sections de route
      ('route') et les valeurs sont des listes de coordonnées (longitude, latitude)
      des bornes supplémentaires à ajouter.
    """

    df_trafic['borne_ids_proches'] = df_trafic['borne_ids_proches'].apply(
        lambda b: ast.literal_eval(b) if isinstance(b, str) else b
    )

    bornes_a_ajouter = {}
    #Itérations sur l'ensemble des sections
    for _, section in df_trafic.iterrows():
        #Prétraitement de la section courante
        section_id = section['route']
        section_geom = section['geometry']

        if isinstance(section_geom, str):
            section_geom = loads(section_geom)
        if not isinstance(section_geom, (LineString, MultiLineString)):
            continue

        borne_ids = section['borne_ids_proches']
        if not isinstance(borne_ids, list):
            continue

        bornes_associees = df_bornes[df_bornes['idBornes'].isin(borne_ids)]
        bornes_coords = [
            (row['consolidated_longitude'], row['consolidated_latitude'])
            for _, row in bornes_associees.iterrows()
        ]
        #Appelle la fonction pour chaque section
        positions = ajouter_bornes_par_section(section_geom, bornes_coords, x, longueur_min)
        if positions:
            bornes_a_ajouter[section_id] = positions

    return bornes_a_ajouter

bornes_a_ajouter = ajouter_bornes(gdf_trafic, gdf_bornes, 60000, 30000)
print(f"Bornes à ajouter : {bornes_a_ajouter}")


Bornes à ajouter : {'A0019': [(690478.5626592279, 6772046.362282833)], 'A0028': [(517533.12685278023, 6891505.043413334)], 'A0065': [(431104.36611996737, 6328761.72078047)], 'A0066': [(592014.7943946114, 6244536.335321331)], 'A0077': [(703829.7235933072, 6673786.732272812)], 'A0089': [(494993.5422297289, 6439863.064325377)], 'N0079': [(811578.2473443664, 6588655.727448893)]}


# Visualisation

In [53]:
#Préparation des données du trafic
df_trafic['geometry'] = df_trafic['geometry'].apply(
    lambda geom: loads(geom) if isinstance(geom, str) else geom
)

gdf_trafic = gpd.GeoDataFrame(df_trafic, geometry=df_trafic['geometry'], crs="EPSG:2154").to_crs("EPSG:4326")

gdf_trafic['borne_ids_proches'] = gdf_trafic['borne_ids_proches'].apply(
    lambda value: ast.literal_eval(value) if isinstance(value, str) and value != '[]' else []
)

#Préparation des données des bornes
gdf_bornes_existantes = df_bornes.copy()
gdf_bornes_existantes['geometry'] = gdf_bornes_existantes.apply(
    lambda row: Point(row['consolidated_longitude'], row['consolidated_latitude']), axis=1
)
gdf_bornes_existantes = gpd.GeoDataFrame(gdf_bornes_existantes, geometry='geometry', crs="EPSG:4326")

borne_ids_proches = set(
    borne_id for borne_list in gdf_trafic['borne_ids_proches'] for borne_id in borne_list
)
gdf_bornes_existantes = gdf_bornes_existantes[gdf_bornes_existantes['idBornes'].isin(borne_ids_proches)]

# Convertir les bornes à ajouter en GeoDataFrame
bornes_a_ajouter_coords = [
    Point(coord) for coords in bornes_a_ajouter.values() for coord in coords
]
gdf_bornes_a_ajouter = gpd.GeoDataFrame(geometry=bornes_a_ajouter_coords, crs="EPSG:2154").to_crs("EPSG:4326")

m = folium.Map(location=[46.603354, 1.888334], zoom_start=6, tiles='OpenStreetMap')

for _, row in gdf_trafic.iterrows():
    if row['geometry'].geom_type == 'LineString':
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometry'].coords],
            color='red', weight=1, opacity=0.7,
            tooltip=f"Route: {row['route']}<br>Longueur: {row['longueur_totale']/1000} km"
        ).add_to(m)
    elif row['geometry'].geom_type == 'MultiLineString':
        for line in row['geometry'].geoms:
            folium.PolyLine(
                locations=[(coord[1], coord[0]) for coord in line.coords],
                color='red', weight=1, opacity=0.7,
                tooltip=f"Route: {row['route']}<br>Longueur: {row['longueur_totale']/1000} km"
            ).add_to(m)

#Ajout des bornes existantes
for _, row in gdf_bornes_existantes.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=0.5, color='green', fill=True, fill_opacity=0.7,
        tooltip=f"Coordonnees: ({round(row['geometry'].y, 2)}, {round(row['geometry'].x, 2)})"
    ).add_to(m)

#Ajout des bornes calculées
for _, row in gdf_bornes_a_ajouter.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=2, color='blue', fill=True, fill_opacity=0.7,
        tooltip=f"Coordonnees: ({round(row['geometry'].y, 2)}, {round(row['geometry'].x, 2)})"
    ).add_to(m)

#Légende HTML
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 220px; height: 90px;
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px; padding: 10px;">
<b>Legende</b><br>
<i style="background:green;color:white;">&nbsp;&nbsp;&nbsp;&nbsp;</i>&nbsp; Bornes existantes<br>
<i style="background:blue;color:white;">&nbsp;&nbsp;&nbsp;&nbsp;</i>&nbsp; Bornes a ajouter<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))
m.add_child(MeasureControl(primary_length_unit='kilometers')) #Permet de réaliser des mesures de distances sur la carte
#m.save("/content/drive/MyDrive/PIP2025_G8/Donnees/DonneesBrutes/Trafic/2019/carte_bornes_axes_60.html")
m


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Carte du trafic par axe routier
#Préparation des données
if gdf_trafic.crs is None:
    gdf_trafic = gpd.GeoDataFrame(
        gdf_trafic,
        geometry=gdf_trafic['geometry'],
        crs="EPSG:2154"
    )
gdf_trafic = gdf_trafic.to_crs("EPSG:4326")

min_tmja = gdf_trafic['tmja_m'].min()
max_tmja = gdf_trafic['tmja_m'].max()
colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_tmja, vmax=max_tmja, caption='Trafic Moyen Journalier Annuel (TMJA)')

m = folium.Map(location=[46.603354, 1.888334], zoom_start=6, tiles='OpenStreetMap')
for _, row in gdf_trafic.iterrows():
    color = colormap(row['tmja_m']) if not pd.isna(row['tmja_m']) else 'gray'
    tmja_arrondi = round(row['tmja_m']) if not pd.isna(row['tmja_m']) else "Inconnu"
    longueur_km = -(-row['longueur_totale'] // 1000) if not pd.isna(row['longueur_totale']) else "Inconnu"

    if row['geometry'].geom_type == 'LineString':
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometry'].coords],
            color=color, weight=3, opacity=0.7,
            tooltip=f"Route: {row['route']}<br>TMJA: {tmja_arrondi}<br>Longueur: {longueur_km} km"
        ).add_to(m)
    elif row['geometry'].geom_type == 'MultiLineString':
        for line in row['geometry'].geoms:
            folium.PolyLine(
                locations=[(coord[1], coord[0]) for coord in line.coords],
                color=color, weight=3, opacity=0.7,
                tooltip=f"Route: {row['route']}<br>TMJA: {tmja_arrondi}<br>Longueur: {longueur_km} km"
            ).add_to(m)

colormap.add_to(m)
m.add_child(MeasureControl(primary_length_unit='kilometers'))

#m.save("/content/drive/MyDrive/PIP2025_G8/Donnees/DonneesBrutes/Trafic/2019/carte_tmja_troncons.html")
m


Output hidden; open in https://colab.research.google.com to view.